In [ ]:
import pandas as pd
import requests
import zipfile
import io
import plotly.graph_objects as go
import streamlit as st
import sys
import numpy as np
import unicodedata

# ====== INICIALIZACIÓN DE PARÁMETROS STREAMLIT ======
COLOR_HOMBRES = "#889064"
COLOR_MUJERES = "#0b0d0e"
FONT_FAMILY = "Aptos Light"
localidad_query = None

if "active_palette" in locals():
    if len(active_palette) >= 2:
        COLOR_HOMBRES = active_palette[1]
    if len(active_palette) >= 1:
        COLOR_MUJERES = active_palette[0]

if "active_font" in locals():
    FONT_FAMILY = active_font

if "LOCALIDAD_SELECCIONADA" in locals():
    localidad_query = LOCALIDAD_SELECCIONADA.strip()

if not localidad_query:
    st.error("❌ Error: No se ha seleccionado una localidad en la aplicación Streamlit.")
    sys.exit(1)


# ===================================================
# ====== CONFIGURACIÓN Y DESCARGA DEL DATASET ======
# ===================================================
URL = "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/iter/iter_00_cpv2020_csv.zip"

rangos = [
    ("0 a 4 años", "P_0A4_F", "P_0A4_M"), ("5 a 9 años", "P_5A9_F", "P_5A9_M"),
    ("10 a 14 años", "P_10A14_F", "P_10A14_M"), ("15 a 19 años", "P_15A19_F", "P_15A19_M"),
    ("20 a 24 años", "P_20A24_F", "P_20A24_M"), ("25 a 29 años", "P_25A29_F", "P_25A29_M"),
    ("30 a 34 años", "P_30A34_F", "P_30A34_M"), ("35 a 39 años", "P_35A39_F", "P_35A39_M"),
    ("40 a 44 años", "P_40A44_F", "P_40A44_M"), ("45 a 49 años", "P_45A49_F", "P_45A49_M"),
    ("50 a 54 años", "P_50A54_F", "P_50A54_M"), ("55 a 59 años", "P_55A59_F", "P_55A59_M"),
    ("60 a 64 años", "P_60A64_F", "P_60A64_M"), ("65 a 69 años", "P_65A69_F", "P_65A69_M"),
    ("70 a 74 años", "P_70A74_F", "P_70A74_M"), ("75 a 79 años", "P_75A79_F", "P_75A79_M"),
    ("80 a 84 años", "P_80A84_F", "P_80A84_M"), ("85 y más", "P_85YMAS_F", "P_85YMAS_M"),
]
columnas_poblacion = ["POBTOT", "POBFEM", "POBMAS"] + [col for _, col_f, col_m in rangos for col in [col_f, col_m]]


# ==========================================================
# 🌟 FUNCIONES DE LIMPIEZA
# ==========================================================

def limpiar_numero(x):
    """Convierte cualquier valor (NaN, texto, *, vacío) a entero seguro (0)."""
    if pd.isna(x):
        return 0
    x = str(x).strip().replace("*", "")
    if x in ["", "-", "N/A", "nan", "None", " "]:
        return 0
    try:
        return int(float(x))
    except:
        return 0


def normalizar(s):
    """Convierte la cadena a minúsculas y quita acentos para búsqueda robusta."""
    if pd.isna(s):
        return ""
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    return s


# ==========================================================
# 🌟 FUNCIÓN DE CARGA DE DATOS (CON FIX ROBUSTO DE ACENTOS)
# ==========================================================

@st.cache_data
def cargar_datos(url):
    st.info("Descargando Censo de Población y Vivienda 2020 (INEGI)...")
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        
        memfile = io.BytesIO(response.content)
        with zipfile.ZipFile(memfile, 'r') as z:
            target_file = None
            for file_name in z.namelist():
                if "conjunto_de_datos_iter" in file_name and file_name.endswith(".csv"):
                    target_file = file_name
                    break
            
            if not target_file:
                raise FileNotFoundError("No se encontró el archivo CSV esperado dentro del ZIP.")

            # 🔥 FIX DECODIFICACIÓN ROBUSTA para evitar Mojibake (Santa MarÃ­a)
            with z.open(target_file) as f:
                raw_bytes = f.read()
            
            try:
                # Intento 1: Decodificar con latin1 (común en datasets de INEGI)
                text = raw_bytes.decode("latin1")
            except:
                # Intento 2: Fallback a UTF-8 con reemplazo si latin-1 falla
                text = raw_bytes.decode("utf-8", errors="replace")

            # Reparar mojibake persistente (convierte latin1 mal interpretado a UTF-8 limpio)
            text = text.encode("latin1", errors="ignore").decode("utf-8", errors="ignore")

            # Leer el archivo ya decodificado como string
            df = pd.read_csv(
                io.StringIO(text), 
                skipinitialspace=True,
                dtype=str, 
                engine="python"
            )
                
    except Exception as e:
        st.error(f"❌ Error al descargar o leer el archivo del INEGI: {e}")
        sys.exit(1)

    # === FIX FINAL DE BOM Y LIMPIEZA ===
    # 1. Limpieza de caracteres BOM/no deseados en columnas
    df.columns = df.columns.str.replace(r"[\ufeffï»¿]+", "", regex=True).str.strip()

    # 2. Limpieza de caracteres BOM en los valores de las celdas (aunque la decodificación ya ayudó)
    for col in ["NOM_LOC", "NOM_MUN", "NOM_ENT"]:
        if col in df.columns:
            df[col] = df[col].str.replace(r"[\ufeffï»¿]+", "", regex=True).str.strip()

    # === Convertir columnas numéricas limpiamente ===
    for col in columnas_poblacion:
        if col in df.columns:
            df[col] = df[col].apply(limpiar_numero)

    df["NOM_LOC_NORMALIZED"] = df["NOM_LOC"].apply(normalizar)
    df["LOC"] = df["LOC"].astype(str).str.strip()

    df_localidades = df[(df["LOC"] != "0") & (df["LOC"] != "9998") & (df["LOC"] != "9999")].copy()
    df_localidades = df_localidades[df_localidades['POBTOT'] > 0].reset_index(drop=True)
    
    return df_localidades


# ==========================================================
# ====== LÓGICA PRINCIPAL ======
# ==========================================================

df_localidades = cargar_datos(URL)

localidad_query_normalized = normalizar(localidad_query)
st.info(f"Buscando localidades que contengan: **'{localidad_query}'**")

resultados = df_localidades[
    df_localidades["NOM_LOC_NORMALIZED"].str.contains(localidad_query_normalized, na=False)
]


# ==========================================================
# 🌟 FUNCIÓN PARA CONSTRUIR PIRÁMIDE
# ==========================================================

def construir_piramide(fila: pd.Series):
    edades = []
    mujeres = []
    hombres = []

    for etiqueta, col_f, col_m in rangos:
        val_f = fila.get(col_f, 0)
        val_m = fila.get(col_m, 0)
        
        edades.append(etiqueta)
        mujeres.append(val_f)
        hombres.append(-val_m)

    fig = go.Figure()

    fig.add_trace(go.Bar(
        y=edades, x=hombres, name="Hombres", orientation="h",
        marker_color=COLOR_HOMBRES,
        hovertemplate="<b>Hombres</b>: %{customdata:,}<extra></extra>",
        customdata=[abs(x) for x in hombres]
    ))

    fig.add_trace(go.Bar(
        y=edades, x=mujeres, name="Mujeres", orientation="h",
        marker_color=COLOR_MUJERES,
        hovertemplate="<b>Mujeres</b>: %{x:,}<extra></extra>"
    ))

    max_val = max(max(mujeres), max(abs(x) for x in hombres))
    step = max(round(max_val / 5), 1)

    left_vals = [-i for i in range(0, max_val + step, step)]
    right_vals = [i for i in range(0, max_val + step, step)]

    tickvals = left_vals + right_vals[1:]
    ticktext = [f"{abs(i):,}" for i in left_vals] + [f"{i:,}" for i in right_vals[1:]]

    fig.update_layout(
        # === CAMBIO 1: Título Centrado ===
        title={
            "text": f"Pirámide Poblacional: {fila['NOM_LOC']}, {fila['NOM_MUN']}, {fila['NOM_ENT']} (2020)",
            "x": 0.5,
            "xanchor": "center"
        },
        barmode="overlay",
        bargap=0.1,
        xaxis=dict(
            title="Población",
            tickvals=tickvals,
            ticktext=ticktext,
            zeroline=True,
            zerolinewidth=1.5,
            zerolinecolor="#666"
        ),
        yaxis=dict(title="Edad"),
        font=dict(family=FONT_FAMILY, size=14),
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5),
        # === CAMBIO 2: Margen Inferior Aumentado ===
        margin=dict(l=40, r=40, t=80, b=150)
    )

    # === CAMBIO 3: Fuente de Datos (Inferior Izquierda) ===
    fig.add_annotation(
        text="Fuente: INEGI (Censo de Población y Vivienda 2020)",
        xref="paper", yref="paper",
        x=0,      # Alineado a la izquierda
        y=-0.3,   # Coordenada negativa, debajo de la leyenda
        showarrow=False,
        xanchor='left',
        yanchor='top',
        font=dict(size=12, color="gray", family=FONT_FAMILY)
    )

    return fig


# ==========================================================
# ====== MOSTRAR RESULTADOS ======
# ==========================================================

if not resultados.empty:
    st.success(f"✅ Se encontraron **{len(resultados)}** coincidencias para '{localidad_query}'.")

    for i, fila in resultados.iterrows():
        pobtot = fila["POBTOT"]
        pobfem = fila["POBFEM"]
        pobmas = fila["POBMAS"]
        
        st.markdown(f"### {fila['NOM_LOC']}, {fila['NOM_MUN']}, {fila['NOM_ENT']}")

        col1, col2, col3 = st.columns(3)
        
        with col1:
            st.metric("Población Total", f"{pobtot:,.0f}")

        if pobfem + pobmas > 0:
            pct_fem = (pobfem / pobtot) * 100
            pct_mas = (pobmas / pobtot) * 100
            
            with col2:
                st.metric("Mujeres", f"{pobfem:,.0f}", f"{pct_fem:.2f}%")
            with col3:
                st.metric("Hombres", f"{pobmas:,.0f}", f"{pct_mas:.2f}%")
        else:
            with col2:
                st.metric("Mujeres", "N/D")
            with col3:
                st.metric("Hombres", "N/D")

        fig = construir_piramide(fila)
        st.plotly_chart(fig, use_container_width=True)

        # ---------------------------------------------------------
        # TABLA DE DATOS (Visible, sin expander)
        # ---------------------------------------------------------
        st.markdown("**Datos detallados por Grupo de Edad**")

        # Construir la lista de datos para la tabla
        datos_tabla = []
        for etiqueta, col_f, col_m in rangos:
            val_f = fila.get(col_f, 0)
            val_m = fila.get(col_m, 0)
            datos_tabla.append({
                "Grupo de Edad": etiqueta,
                "Mujeres": val_f,
                "Hombres": val_m,
                "Total": val_f + val_m
            })
        
        # Crear DataFrame y mostrarlo
        df_tabla = pd.DataFrame(datos_tabla)
        
        st.dataframe(
            df_tabla,
            use_container_width=True,
            hide_index=True,
            column_config={
                "Grupo de Edad": st.column_config.TextColumn("Grupo de Edad"),
                "Mujeres": st.column_config.NumberColumn(format="%,d"),
                "Hombres": st.column_config.NumberColumn(format="%,d"),
                "Total": st.column_config.NumberColumn(format="%,d"),
            }
        )

        st.markdown("---")

else:
    st.error(f"❌ No se encontraron localidades que contengan: **{localidad_query}**.")
    st.info("Intenta con un nombre de localidad más general o diferente.")